<a href="https://colab.research.google.com/github/tumi2003/hello-world/blob/main/ENDG_511_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Dataset into a pandas Dataframe

In [20]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Load all dataset files
files = ["/content/drive/MyDrive/UNSW-NB15_1.csv", "/content/drive/MyDrive/UNSW-NB15_2.csv", "/content/drive/MyDrive/UNSW-NB15_3.csv", "/content/drive/MyDrive/UNSW-NB15_4.csv"]
df_list = [pd.read_csv(f) for f in files]

# Combine all parts into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

# Display dataset info
print(df.info())
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-20-82e464ca3e7d>:8: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f) for f in files]
<ipython-input-20-82e464ca3e7d>:8: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f) for f in files]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540047 entries, 0 to 2540046
Data columns (total 49 columns):
 #   Column            Dtype  
---  ------            -----  
 0   srcip             object 
 1   sport             object 
 2   dstip             object 
 3   dsport            object 
 4   proto             object 
 5   state             object 
 6   dur               float64
 7   sbytes            int64  
 8   dbytes            int64  
 9   sttl              int64  
 10  dttl              int64  
 11  sloss             int64  
 12  dloss             int64  
 13  service           object 
 14  Sload             float64
 15  Dload             float64
 16  Spkts             int64  
 17  Dpkts             int64  
 18  swin              int64  
 19  dwin              int64  
 20  stcpb             int64  
 21  dtcpb             int64  
 22  smeansz           int64  
 23  dmeansz           int64  
 24  trans_depth       int64  
 25  res_bdy_len       int64  
 26  Sjit          

Fill in Missing Values


In [21]:
# Check for missing values
print(df.isnull().sum())

# Fill or drop missing values (if any)
df.fillna(0, inplace=True)


srcip                     0
sport                     0
dstip                     0
dsport                    0
proto                     0
state                     0
dur                       0
sbytes                    0
dbytes                    0
sttl                      0
dttl                      0
sloss                     0
dloss                     0
service                   0
Sload                     0
Dload                     0
Spkts                     0
Dpkts                     0
swin                      0
dwin                      0
stcpb                     0
dtcpb                     0
smeansz                   0
dmeansz                   0
trans_depth               0
res_bdy_len               0
Sjit                      0
Djit                      0
Stime                     0
Ltime                     0
Sintpkt                   0
Dintpkt                   0
tcprtt                    0
synack                    0
ackdat                    0
is_sm_ips_ports     

In [22]:
print(df.columns)  # Display all column names


Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')


In [23]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
categorical_columns = ['srcip','proto', 'service', 'state','dstip']  # Example categorical features
encoder = LabelEncoder()

for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col])


In [24]:
from sklearn.preprocessing import MinMaxScaler

# Select numeric features (excluding label)
features = df.drop(columns=['Label','attack_cat','Stime','Ltime','sport','dsport','ct_ftp_cmd'])  # 'label' is the attack/normal classification column


# Normalize features
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(features)

# Convert back to DataFrame
df_scaled = pd.DataFrame(df_scaled, columns=features.columns)


In [ ]:
from sklearn.model_selection import train_test_split

# Define X (features) and y (labels)
X = df_scaled
y = df['Label']  # '1' for attack, '0' for normal

# Split dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (2032037, 42)
Testing set size: (508010, 42)


In [25]:
from sklearn.ensemble import IsolationForest

# Define and train model
iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
iso_forest.fit(X_train)

# Predict anomalies (-1 = anomaly, 1 = normal)
y_pred = iso_forest.predict(X_test)

# Convert predictions to binary format (1 = anomaly, 0 = normal)
y_pred_binary = [1 if pred == -1 else 0 for pred in y_pred]


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define input shape
input_dim = X_train.shape[1]

# Build Autoencoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(input_dim, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse")

# Train Autoencoder
autoencoder.fit(X_train, X_train, epochs=20, batch_size=32, validation_data=(X_test, X_test))


Epoch 1/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 179s 3ms/step - loss: 0.0014 - val_loss: 5.7469e-05
Epoch 2/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 177s 3ms/step - loss: 3.5032e-05 - val_loss: 3.1311e-05
Epoch 3/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 177s 3ms/step - loss: 2.7589e-05 - val_loss: 2.5834e-05
Epoch 4/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 183s 3ms/step - loss: 2.4826e-05 - val_loss: 2.4567e-05
Epoch 5/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 185s 3ms/step - loss: 2.3282e-05 - val_loss: 2.5172e-05
Epoch 6/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 186s 3ms/step - loss: 2.1507e-05 - val_loss: 1.9739e-05
Epoch 7/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 191s 3ms/step - loss: 1.9650e-05 - val_loss: 2.0682e-05
Epoch 8/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 184s 3ms/step - loss: 1.9053e-05 - val_loss: 1.3717e-05
Epoch 9/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 206s 3ms/step - loss: 1.2554e-05 - val_loss: 9.6822e-06
Epoch 10/20
63502/63502 ━━━━━━━━━━━━━━━━━━━━ 193s 3ms/step - loss: 1.1546e-05 - val_loss: 9.8715e-06
Epo

In [27]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate Isolation Forest
print("Isolation Forest Results:")
print(classification_report(y_test, y_pred_binary))

# Evaluate Autoencoder (Reconstruction Error)
reconstructions = autoencoder.predict(X_test)
mse = ((X_test - reconstructions) ** 2).mean(axis=1)

# Set threshold for anomaly detection
threshold = mse.mean() + 2 * mse.std()
y_pred_autoencoder = [1 if error > threshold else 0 for error in mse]

print("Autoencoder Results:")
print(classification_report(y_test, y_pred_autoencoder))


Isolation Forest Results:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92    443753
           1       0.23      0.09      0.13     64257

    accuracy                           0.85    508010
   macro avg       0.56      0.52      0.52    508010
weighted avg       0.80      0.85      0.82    508010

15876/15876 ━━━━━━━━━━━━━━━━━━━━ 22s 1ms/step
Autoencoder Results:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93    443753
           1       0.51      0.00      0.01     64257

    accuracy                           0.87    508010
   macro avg       0.69      0.50      0.47    508010
weighted avg       0.83      0.87      0.82    508010



In [28]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Convert Isolation Forest output (-1 = anomaly, 1 = normal) to (1 = anomaly, 0 = normal)
y_pred = [1 if pred == -1 else 0 for pred in iso_forest.predict(X_test)]

# Compute Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8468
Confusion Matrix:
[[424219  19534]
 [ 58280   5977]]


In [29]:
import numpy as np

# Get model reconstructions
reconstructions = autoencoder.predict(X_test)

# Compute Mean Squared Error (MSE)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

# Set threshold (mean + 2 * standard deviation)
threshold = mse.mean() + 2 * mse.std()

# Predict anomalies
y_pred_autoencoder = [1 if error > threshold else 0 for error in mse]

# Compute Accuracy
accuracy = accuracy_score(y_test, y_pred_autoencoder)
print(f"Autoencoder Accuracy: {accuracy:.4f}")


15876/15876 ━━━━━━━━━━━━━━━━━━━━ 22s 1ms/step
Autoencoder Accuracy: 0.8735


In [30]:
import joblib

# Save the trained Isolation Forest model
joblib.dump(iso_forest, "isolation_forest_model.pkl")

print("Model saved as isolation_forest_model.pkl")


Model saved as isolation_forest_model.pkl


In [31]:
# Save the Autoencoder model in TensorFlow format
autoencoder.save("autoencoder_model.h5")

print("Model saved as autoencoder_model.h5")


Model saved as autoencoder_model.h5


In [32]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load trained Isolation Forest model
iso_forest = joblib.load("isolation_forest_model.pkl")

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    features = data['features']

    # Make prediction
    prediction = iso_forest.predict([features])
    anomaly = 1 if prediction[0] == -1 else 0

    return jsonify({'anomaly': anomaly})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
